In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

 99% 746M/750M [00:06<00:00, 135MB/s]
100% 750M/750M [00:06<00:00, 126MB/s]


In [ ]:
!unzip stanford-dogs-dataset.zip 'images/Images/n02085620-Chihuahua/*' 'images/Images/n02085782-Japanese_spaniel/*' 'images/Images/n02088632-bluetick/*' 'images/Images/n02088238-basset/*' 'images/Images/n02088364-beagle/*' 'images/Images/n02087046-toy_terrier/*' 'images/Images/n02086240-Shih-Tzu/*' 'images/Images/n02087394-Rhodesian_ridgeback/*' -d '/content/Images'

Archive:  stanford-dogs-dataset.zip
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_10074.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_10131.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_10621.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_1073.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_10976.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_11140.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_11238.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_11258.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_11337.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_11477.jpg  
  inflating: /content/Images/images/Images/n02085620-Chihuahua/n02085620_1152.jpg  
  inflating: /content/Images/im

In [ ]:
import tensorflow as tf
import os
import random


path = "/content/Images/images/Images"
categories = ["n02085620-Chihuahua", "n02085782-Japanese_spaniel",
              "n02086240-Shih-Tzu", "n02087046-toy_terrier",
              "n02087394-Rhodesian_ridgeback", "n02088238-basset",
              "n02088364-beagle", "n02088632-bluetick"]
data = []
for category in categories:
    folder = os.path.join(path, category)
    files = os.listdir(folder)
    for file in files:
        data.append((os.path.join(folder, file), categories.index(category)))

random.shuffle(data)

def read_image(filename, label):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    return image, label

train_size = int(0.7 * len(data))
val_size = int(0.2 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

def read_image(filename, label):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    return image, label

In [ ]:
image_size = (224, 224)
batch_size = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=0.2,
    subset='training',
    seed=1337
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=0.2,
    subset='validation',
    seed=1337
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False
)

Found 1436 files belonging to 8 classes.
Using 1149 files for training.
Found 1436 files belonging to 8 classes.
Using 287 files for validation.
Found 1436 files belonging to 8 classes.


In [ ]:
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break

data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 8)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False,
                      input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(8, activation='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy',
             metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint('stan_dogs_resnet.h5',
                                  save_best_only=True)
]
history = model.fit(train_dataset,
                   epochs=15,
                   validation_data=validation_dataset,
                   callbacks=callbacks)

Epoch 1/15
36/36 [==============================] - ETA: 0s - loss: 1.4374 - accuracy: 0.6510

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 17s 329ms/step - loss: 1.4374 - accuracy: 0.6510 - val_loss: 0.4989 - val_accuracy: 0.8223
Epoch 2/15
36/36 [==============================] - 6s 169ms/step - loss: 0.2909 - accuracy: 0.8982 - val_loss: 0.4940 - val_accuracy: 0.8049
Epoch 3/15
36/36 [==============================] - 5s 142ms/step - loss: 0.1649 - accuracy: 0.9504 - val_loss: 0.5170 - val_accuracy: 0.8293
Epoch 4/15
36/36 [==============================] - 6s 158ms/step - loss: 0.1061 - accuracy: 0.9652 - val_loss: 0.4870 - val_accuracy: 0.8328
Epoch 5/15
36/36 [==============================] - 6s 157ms/step - loss: 0.0427 - accuracy: 0.9913 - val_loss: 0.4393 - val_accuracy: 0.8571
Epoch 6/15
36/36 [==============================] - 5s 140ms/step - loss: 0.0325 - accuracy: 0.9956 - val_loss: 0.4836 - val_accuracy: 0.8641
Epoch 7/15
36/36 [==============================] - 5s 141ms/step - loss: 0.0138 - accuracy: 0.9991 - val_loss: 0.4408 - val_accuracy: 0.8606
Epoch 8/15
36/36

In [ ]:
test_model = keras.models.load_model('stan_dogs_resnet.h5')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

45/45 [==============================] - 7s 133ms/step - loss: 0.1071 - accuracy: 0.9701
Test accuracy: 0.970


In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),])


inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = keras.applications.resnet.preprocess_input(x)
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions =layers.Dense(8, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy',
             metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint('stan_dogs_resnet2.h5',
                                  save_best_only=True)
]
history = model.fit(train_dataset,
                   epochs=15,
                   validation_data=validation_dataset,
                   callbacks=callbacks)

Epoch 1/15
36/36 [==============================] - ETA: 0s - loss: 1.1667 - accuracy: 0.6266

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 24s 309ms/step - loss: 1.1667 - accuracy: 0.6266 - val_loss: 0.2690 - val_accuracy: 0.9164
Epoch 2/15
36/36 [==============================] - 6s 157ms/step - loss: 0.4473 - accuracy: 0.8520 - val_loss: 0.2099 - val_accuracy: 0.9094
Epoch 3/15
36/36 [==============================] - 6s 163ms/step - loss: 0.3215 - accuracy: 0.8877 - val_loss: 0.1735 - val_accuracy: 0.9199
Epoch 4/15
36/36 [==============================] - 7s 188ms/step - loss: 0.3119 - accuracy: 0.8790 - val_loss: 0.1838 - val_accuracy: 0.9268
Epoch 5/15
36/36 [==============================] - 6s 146ms/step - loss: 0.2577 - accuracy: 0.9034 - val_loss: 0.2606 - val_accuracy: 0.9129
Epoch 6/15
36/36 [==============================] - 6s 155ms/step - loss: 0.2316 - accuracy: 0.9060 - val_loss: 0.2081 - val_accuracy: 0.9233
Epoch 7/15
36/36 [==============================] - 6s 148ms/step - loss: 0.1808 - accuracy: 0.9373 - val_loss: 0.2285 - val_accuracy: 0.9199
Epoch 8/15
36/36

In [ ]:
test_model = keras.models.load_model('stan_dogs_resnet2.h5')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

45/45 [==============================] - 6s 120ms/step - loss: 0.1387 - accuracy: 0.9492
Test accuracy: 0.949
